# Digit Recognizer

## Goal

Idnetify digits from handwritten images.

## Setup

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm_notebook

In [28]:
#Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Check cuda availability
device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))

## Data Preprocessing

In [29]:
# Split data into training set and validation set
train = train.to_numpy()
test = test.to_numpy()
split = len(train)*4//5
X_train = train[:split,1:]
y_train = train[:split,0]
X_val = train[split:,1:]
y_val = train[split:,0]

# Transform numpy array to torch tensor
X_train = torch.tensor(X_train.astype(float))
y_train = torch.tensor(y_train.astype(float))
X_val = torch.tensor(X_val.astype(float))
y_val = torch.tensor(y_val.astype(float))
test = torch.tensor(test.astype(float))

# Reshape data
X_train = X_train.view(-1, 28, 28).unsqueeze(1)
X_val = X_val.view(-1, 28, 28).unsqueeze(1)
test = test.view(-1, 28, 28).unsqueeze(1)

# Normalize data
X_train = (X_train-X_train.mean())/X_train.std()
X_val = (X_val-X_val.mean())/X_val.std()
test = (test - test.mean())/test.std()

# Cast data to other data types
X_train = X_train.float()
y_train = y_train.long()
X_val = X_val.float()
y_val = y_val.long()
test = test.float()

## Model

In [25]:
# Define network subclass
# After parameter tuning, set number of channels 3.
class Net(nn.Module):
    
    # c: number of channels
    # r: ratio to define dimension of output in hidden layers
    def __init__(self, c=12):
        super().__init__()
        self.c = c
        self.conv1 = nn.Conv2d(1, c, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(c, c, kernel_size = 3, padding = 1)
        self.fc1 = nn.Linear(7*7*(c), 7*7*(c))
        self.fc2 = nn.Linear(7*7*(c), 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 7*7*(self.c))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Training

In [26]:
def training_loop(epochs, optimizer, model, loss_f, train_loader):
    for epoch in tqdm_notebook(range(epochs), position=0, leave=True):
        loss_train = 0.0
        for x, y in train_loader:
            out = model(x.to(device=device))
            loss = loss_f(out, y.to(device=device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        if epoch % 50 == 0 or epoch == epochs - 1:
            print(epoch, loss_train/len(train_loader))

In [30]:
# Create data set variable
ds = TensorDataset(X_train, y_train)
dl = DataLoader(ds, batch_size=1050, shuffle=True)

# Training model
# After parameter tuning, set learning rate 0.006, number of epochs 150
model = Net()
optim = torch.optim.Adam(model.parameters(), lr=0.006)
loss = nn.CrossEntropyLoss()
training_loop(150, optim, model.to(device), loss, dl)

correct = 0
for i in range(len(X_val)):
    ans = model(X_val[i].unsqueeze(0).to(device))
    if torch.argmax(ans) == y_val[i]:
        correct += 1
print(correct/len(X_val))

0 0.5303895631805062
50 5.021021287632266e-06
100 1.317615890172874e-06
149 5.008171362064218e-07

0.9902380952380953


## Results

In [56]:
predict = []
ind = []
for i in range(len(test)):
    ind.append(i+1)
    ans = model(test[i].unsqueeze(0).to(device))
    predict.append(torch.argmax(ans).item())

In [66]:
result = pd.DataFrame({"ImageId": ind, "Label": predict})
result.to_csv("result.csv", index=False)